In [1]:
import numpy as np
# constant size
size = 1000
# Set p(H) apriori for simulation as a biased coin.
p_h = 0.7

# Model experiment as a single biased coin flipped 1000 times.
data = np.random.binomial(1, p_h, size=size)

In [8]:
data

array([1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,

In [2]:
n_states = 2
n_obs = 2

emission = np.array([[0.2,0.8],[0.3,0.7]])
transition = np.array([[0.1,0.9],[0.9,0.1]])


forward = np.random.rand(size,n_states)
scale_factors = np.zeros((size))
forward_hat = np.zeros((size,n_states))

backward = np.random.rand(size,n_states)
backward_hat = np.zeros((size,n_states))

init_prob = np.array([0.5,0.5])
init_obs = np.zeros(n_states)

p_old = -10000
tol = 0.001
max_iter = 500

In [3]:
for ite in range(max_iter):
    forward[0,:] = init_prob * emission[:,data[0]]
    scale_factors[0] = np.sum(forward[0,:])
    forward_hat[0,:] = forward[0,:]/scale_factors[0]
    
    for t in range(size-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[:,data[t]]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        forward[t+1,:] = forward_hat[t+1,:]*np.prod(scale_factors[0:t+1])

    backward[-1,:] = 1
    backward_hat[-1,:] = backward[-1,:]
    for t in reversed(range(size-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],np.transpose(transition))
        backward_hat[t,:] = temp/scale_factors[t+1]
        backward[t,:] = backward_hat[t,:]*np.prod(scale_factors[t+1:-1])


    a = np.zeros((size,n_states))
    b = np.zeros((size,n_states,n_states))
    for i in range(size):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
    for t in range(size-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_obs):
            emission[i,j] = np.sum(a[np.argwhere(data==j),i]) / np.sum(a[:,i])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p}')
    # if p - p_old < tol:
    #     break
    p_old = p

p is:-619.3186171530906
p is:-752.514897040742
p is:-618.4101106315288
p is:-752.5148919031649
p is:-618.4101064031512
p is:-752.5148907903715
p is:-618.4101063272994
p is:-752.51488972839
p is:-618.410106251524
p is:-752.5148886614543
p is:-618.4101061753804
p is:-752.5148875895351
p is:-618.4101060988669
p is:-752.5148865126102
p is:-618.4101060219816
p is:-752.5148854306561
p is:-618.4101059447225
p is:-752.51488434365
p is:-618.4101058670876
p is:-752.5148832515689
p is:-618.4101057890757
p is:-752.5148821543889
p is:-618.4101057106843
p is:-752.5148810520868
p is:-618.4101056319118
p is:-752.5148799446397
p is:-618.4101055527562
p is:-752.5148788320232
p is:-618.4101054732157
p is:-752.5148777142139
p is:-618.4101053932884
p is:-752.514876591188
p is:-618.4101053129721
p is:-752.5148754629214
p is:-618.4101052322651
p is:-752.5148743293901
p is:-618.4101051511655
p is:-752.51487319057
p is:-618.4101050696713
p is:-752.5148720464368
p is:-618.4101049877804
p is:-752.5148708969664
p

In [7]:
transition

array([[0.10156675, 0.89843325],
       [0.9028394 , 0.0971606 ]])

In [5]:
# transition_est = np.random.rand(size,n_states,n_states)
# for t in range(size-1):
#     for i in range(n_states):
#         for j in range(n_states):
#             transition_est[t,i,j] = forward[t,i]*transition[i,j]*emission[j,data[t+1]]*backward[t+1,j] / (np.sum(forward[t,:]*backward[t,:]))
